<a href="https://colab.research.google.com/github/AndreGalvao13/Faturamento_Baanko_2024/blob/main/Tentativa_de_cria%C3%A7%C3%A3o_de_Dashboard_interativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Importando as bibliotecas necessárias
import pandas as pd
from google.colab import files
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from IPython.display import display

# Passo 1: Fazer upload das duas planilhas
uploaded = files.upload()

# Passo 2: Carregar as tabelas automaticamente a partir dos arquivos enviados
tabela1 = pd.read_excel(next(iter(uploaded)))
uploaded = files.upload()  # Solicitar novo upload para a segunda planilha
tabela2 = pd.read_excel(next(iter(uploaded)))

# Passo 3: Concatenar as duas tabelas
tabela_unificada = pd.concat([tabela1, tabela2], axis=0, ignore_index=True)

# Passo 4: Definir as colunas a serem mantidas
colunas_para_manter = ['Nome do cliente', 'Data prevista', 'Valor recebido da parcela (R$)', 'Categoria 1', 'Centro de Custo 1']

# Manter apenas as colunas desejadas
tabela_unificada = tabela_unificada[colunas_para_manter]

# Converter a coluna 'Data prevista' para formato de data
tabela_unificada['Data prevista'] = pd.to_datetime(tabela_unificada['Data prevista'], errors='coerce')

# Criar a aplicação Dash
app = Dash(__name__)

# Layout do dashboard
app.layout = html.Div([
    html.H1("Dashboard de Faturamento - Análise por Período, Centro de Custo e Categoria", id='titulo', style={'textAlign': 'center'}),
    html.H3(id='subtitulo', style={'textAlign': 'center'}),

    # Filtro para o período
    dcc.DatePickerRange(
        id='data-picker',
        start_date=tabela_unificada['Data prevista'].min().date(),
        end_date=tabela_unificada['Data prevista'].max().date(),
        display_format='DD/MM/YYYY',
        style={'padding': '10px'}
    ),

    # Filtro para a categoria financeira
    dcc.Dropdown(
        id='categoria-dropdown',
        options=[{'label': categoria, 'value': categoria} for categoria in tabela_unificada['Categoria 1'].unique()],
        multi=True,
        placeholder="Selecione as categorias financeiras",
        style={'width': '50%'}
    ),

    # Filtro para o centro de custo
    dcc.Dropdown(
        id='centro-custo-dropdown',
        options=[{'label': centro, 'value': centro} for centro in tabela_unificada['Centro de Custo 1'].unique()],
        multi=True,
        placeholder="Selecione os centros de custo",
        style={'width': '50%'}
    ),

    # Gráfico de barras - Faturamento por Centro de Custo
    dcc.Graph(id='grafico-barras-centro-custo'),

    # Gráfico de barras - Faturamento por Categoria
    dcc.Graph(id='grafico-barras-categoria'),

    # Gráfico de pizza - Faturamento por Centro de Custo
    dcc.Graph(id='grafico-pizza-centro-custo'),

    # Gráfico de pizza - Faturamento por Categoria
    dcc.Graph(id='grafico-pizza-categoria')
])

# Função para atualizar os gráficos com base nos filtros
@app.callback(
    [Output('grafico-barras-centro-custo', 'figure'),
     Output('grafico-barras-categoria', 'figure'),
     Output('grafico-pizza-centro-custo', 'figure'),
     Output('grafico-pizza-categoria', 'figure'),
     Output('subtitulo', 'children')],  # Atualizando o subtítulo
    [Input('data-picker', 'start_date'),
     Input('data-picker', 'end_date'),
     Input('categoria-dropdown', 'value'),
     Input('centro-custo-dropdown', 'value')]
)
def update_graphs(start_date, end_date, categorias, centros_custo):
    # Filtrar os dados com base nos inputs
    tabela_filtrada = tabela_unificada[
        (tabela_unificada['Data prevista'] >= start_date) &
        (tabela_unificada['Data prevista'] <= end_date)
    ]

    if categorias:
        tabela_filtrada = tabela_filtrada[tabela_filtrada['Categoria 1'].isin(categorias)]

    if centros_custo:
        tabela_filtrada = tabela_filtrada[tabela_filtrada['Centro de Custo 1'].isin(centros_custo)]

    # Calcular o faturamento total do período selecionado
    faturamento_total = tabela_filtrada['Valor recebido da parcela (R$)'].sum()

    # Gráfico de barras - Faturamento por Centro de Custo
    faturamento_barras_centro = tabela_filtrada.groupby('Centro de Custo 1').agg({'Valor recebido da parcela (R$)': 'sum'}).reset_index()
    fig_barras_centro = px.bar(faturamento_barras_centro, x='Centro de Custo 1', y='Valor recebido da parcela (R$)',
                               title='Faturamento por Centro de Custo', labels={'Valor recebido da parcela (R$)': 'Faturamento'},
                               color='Centro de Custo 1')  # Adicionando cores com base no Centro de Custo

    # Gráfico de barras - Faturamento por Categoria
    faturamento_barras_categoria = tabela_filtrada.groupby('Categoria 1').agg({'Valor recebido da parcela (R$)': 'sum'}).reset_index()
    fig_barras_categoria = px.bar(faturamento_barras_categoria, x='Categoria 1', y='Valor recebido da parcela (R$)',
                                  title='Faturamento por Categoria', labels={'Valor recebido da parcela (R$)': 'Faturamento'},
                                  color='Categoria 1')  # Adicionando cores com base na Categoria

    # Gráfico de pizza - Faturamento por Centro de Custo
    faturamento_pizza_centro = tabela_filtrada.groupby('Centro de Custo 1').agg({'Valor recebido da parcela (R$)': 'sum'}).reset_index()
    fig_pizza_centro = px.pie(faturamento_pizza_centro, names='Centro de Custo 1', values='Valor recebido da parcela (R$)',
                               title='Participação de cada Centro de Custo no Faturamento Total')

    # Gráfico de pizza - Faturamento por Categoria
    faturamento_pizza_categoria = tabela_filtrada.groupby('Categoria 1').agg({'Valor recebido da parcela (R$)': 'sum'}).reset_index()
    fig_pizza_categoria = px.pie(faturamento_pizza_categoria, names='Categoria 1', values='Valor recebido da parcela (R$)',
                                  title='Participação de cada Categoria no Faturamento Total')

    # Subtítulo com faturamento total
    subtitulo = f"Faturamento Total no Período Selecionado: R$ {faturamento_total:,.2f}"

    return fig_barras_centro, fig_barras_categoria, fig_pizza_centro, fig_pizza_categoria, subtitulo

# Rodar a aplicação
if __name__ == '__main__':
    app.run(debug=True)

Saving visao_contas_a_receber bko 2024.xls to visao_contas_a_receber bko 2024 (15).xls


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default



Saving visao_contas_a_receber cbk 2024.xls to visao_contas_a_receber cbk 2024 (15).xls


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default



<IPython.core.display.Javascript object>